In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My\ Drive/Fake\ Orders/

/content/drive/My Drive/Fake Orders


In [3]:
ls

 20191207_spaces_info.csv  'Fake Orders empty time.csv'
 20191209_Real_order.csv   'Fake Orders in Real orders.csv'


## Read the Orders & Spaces inforomation

In [4]:
import csv
import datetime

##### Read the spaces information #####

with open('20191207_spaces_info.csv', newline='', encoding='big5') as space_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  spaces_info = csv.DictReader(space_file)
  
  # 以迴圈讀取指定欄位
  locate_id = []; space_id = []; price = []; size = []; zipcode = [];
  open_day = []; close_day = []; open_time = []; close_time = []; creat_time = []; mrt = [];
  for i in spaces_info:
    locate_id.append(i['locate_id'])
    space_id.append(i['space_id'])
    price.append(i['price'])
    size.append(i['size'])
    zipcode.append(i['zipcode'])
    open_day.append(i['open_day'])
    close_day.append(i['close_day'])
    open_time.append(i['open_time'])
    close_time.append(i['close_time'])
    creat_time.append(i['creat_time'])
    mrt.append(i['mrt'])
  space_file.close()

space_id = list(map(int, space_id))
price = list(map(int, price))
size = list(map(int, size))
zipcode = list(map(int, zipcode))
open_day = list(map(int, open_day))
close_day = list(map(int, close_day))

print("Read the Space information successfully.")

##### Read the real Orders information #####

with open('20191209_Real_order.csv', newline='', encoding='big5') as test_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  test_info = csv.DictReader(test_file)
  
  # 以迴圈讀取指定欄位
  r_space_id = []; r_date = []; r_start = []; r_end = [];
  r_start_time_group = []; r_end_time_group = [];
  for i in test_info:
    r_space_id.append(i['space_id'])
    r_date.append(i['date'])
    r_start.append(i['start'])
    r_end.append(i['end'])
    r_start_time_group.append(i['start_time_group'])
    r_end_time_group.append(i['End_time_group'])
  test_file.close()

r_space_id = list(map(int, r_space_id))
r_no_dupl_space_id = list(set(r_space_id)) #delete the duplicates & use set() to sort the 
r_start_time_group = list(map(int, r_start_time_group))
r_end_time_group = list(map(int, r_end_time_group))

for i in range(0, len(r_date)):
  r_date[i] = datetime.datetime.strptime(r_date[i], "%d/%m/%Y")

print("Read the Real Order information successfully.")

Read the Space information successfully.
Read the Real Order information successfully.


## Generate Fake Orders

In [5]:
format = '%H:%M'
##### Generate the Fake Orders #####
with open('Fake Orders in Real orders.csv', mode='w', newline='', encoding='big5') as f:
    FOrders_writer = csv.writer(f)

    order_id = 1    #訂單編號
    category = 1    #普通方案
    Is_holiday = 0  #是否為假日
    status = 1      #沒租出去

    start_time_group = 0
    end_time_group = 0
    morning = datetime.datetime.strptime("12:00", format)     # 1: 06:00 ~ 12:00
    noon = datetime.datetime.strptime("14:00", format)        # 2: 12:00 ~ 14:00
    afternoon = datetime.datetime.strptime("18:00", format)   # 3: 14:00 ~ 18:00
    night = datetime.datetime.strptime("23:59", format)       # 4: 18:00 ~ 24:00 , 00:00 ~ 06:00
    midnight = datetime.datetime.strptime("06:00", format)

    ##### Write the Fake Orders in real order time #####
    FOrders_writer.writerow(['id', 'locate_id', 'space_id', 'hours', 'price', 'order_price', 'size', 'price_per_size', 'date', 'start', 'end', 'start_time_group', 'end_time_group', 'weekday', 'Is_holiday', 'zipcode', 'category', 'mrt', 'status'])
    total = 4
    count_temp = 0
    len_count = 0
    for i in range(0, len(r_no_dupl_space_id)):  #遍歷真實訂單中的space_id（不重複）
      t = i
      while(r_no_dupl_space_id[i] != space_id[t]):
        t += 1 

      ##### 計算space的創造時間 ######
      len_of_same_space_order = r_space_id.count(r_space_id[len_count]) #計算同一間房間有多少訂單
      len_count += len_of_same_space_order
      
      same_space_time_group = []
      temp_compare_time_group = []

      ##### 設定房間起始日期 #####
      order_start = datetime.datetime.strptime(open_time[t], format) #訂單開始時間
      order_end = datetime.datetime.strptime(close_time[t], format)  #訂單結束時間
      if (order_start >= midnight and order_start < morning):
        temp_compare_time_group.append(1)
      elif (order_start >= morning and order_start < noon):
        temp_compare_time_group.append(2)
      elif (order_start >= noon and order_start < afternoon):
        temp_compare_time_group.append(3)
      else:
        temp_compare_time_group.append(4)
      
      if (order_end > midnight and order_end <= morning):
        temp_compare_time_group.append(1)
      elif (order_end > morning and order_end <= noon):
        temp_compare_time_group.append(2)
      elif (order_end > noon and order_end <= afternoon):
        temp_compare_time_group.append(3)
      else:
        temp_compare_time_group.append(4)
      temp_compare_time_group = list(range(temp_compare_time_group[0], temp_compare_time_group[1] + 1))
      #temp_compare_time_group = [1, 2, 3, 4]

      for g in range(0, len_of_same_space_order):
        if((count_temp + 1) == len(r_date)):
          r_date.append('')
        if(r_date[count_temp] == r_date[count_temp+1]):
          for u in range(r_start_time_group[count_temp], r_end_time_group[count_temp] + 1):
            same_space_time_group.append(u)

        elif(r_date[count_temp] != r_date[count_temp+1]):
          for u in range(r_start_time_group[count_temp], r_end_time_group[count_temp] + 1):
            same_space_time_group.append(u)

          diff_time_group = list(set(temp_compare_time_group) - set(same_space_time_group))
          for k in diff_time_group:
            if (k == 1):
              order_start = datetime.datetime.strptime("09:00", format)
              order_end = morning
            elif (k == 2):
              order_start = morning
              order_end = noon
            elif (k == 3):
              order_start = noon
              order_end = afternoon
            else:
              order_start = afternoon
              order_end = datetime.datetime.strptime("21:00", format)
            ##### 計算訂單租用時間 #####
            diff = order_end - order_start 
            seconds = diff.total_seconds()
            hours = seconds // 3600
            minutes = (seconds % 3600) // 60
            duration = hours + minutes/60  #訂單租用時間
            Is_holiday = 1 if(r_date[count_temp].isoweekday() == 6 or r_date[count_temp].isoweekday() == 7) else 0
            ##### 開始產生訂單 #####
            FOrders_writer.writerow([order_id, locate_id[t], space_id[t], duration, price[t], price[t]*duration, size[t], price[t]/size[t], r_date[count_temp].date().strftime("%d/%m/%Y"), order_start.strftime("%H:%M:%S"), order_end.strftime("%H:%M:%S"), k, k, r_date[count_temp].isoweekday(), Is_holiday, zipcode[t], category, mrt[t], status])
            order_id += 1
          same_space_time_group = []
        count_temp += 1

    f.close()
print("Write the Fake Orders successfully!")

Write the Fake Orders successfully!


## Original Order Format

In [0]:
id	space_id	size	hours	price	order_price	price_per_size	start_time_group	End_time_group	weekday	Is_holiday	country_holiday	zipcode	category
9615	1	12	3	300	900	25	0	1	3	0	0	100	1
9617	1	12	4	300	1200	25	4	4	5	0	0	100	1
9618	1	12	4	300	1200	25	4	4	6	1	0	100	1

## Code Test

In [0]:
same_space_time_group.append(r_start_time_group[count_temp])
same_space_time_group.append(r_end_time_group[count_temp])
same_space_time_group = list(range(same_space_time_group[0], same_space_time_group[1]))

a = 4
b = 5

c = list(range(a, b))
print(c)

d = [1, 3, 1, 2, 3, 2]

print(max(d), '  ', min(d))
d = list(range(min(d), max(d) + 1))
print(d)



In [0]:
print(same_space_time_group)
print(temp_compare_time_group)
print(same_space_time_group_no_dupl)
print(diff_time_group)
print(len_of_same_space_order)
print(count_temp)
print(len_count)
print(t)

print(len(r_date))
print(r_date[-1])

for test in range (10, 15):
  print(test)

f.close()


In [0]:
# print(r_space_id[457])
# print(r_no_dupl_space_id[0])
print(list(set(r_space_id)))
print(space_id)

print(space_id[176])
# print(r_no_dupl_space_id[1])
# len_of_same_space_order = r_space_id.count(r_space_id[459])
# print(len_of_same_space_order)

# len_count = 0
# for i in r_no_dupl_space_id:
#   len_of_same_space_order = r_space_id.count(r_space_id[len_count])
#   len_count += len_of_same_space_order
#   print(len_of_same_space_order)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 31, 41, 51, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 104, 105, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 184]
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 31, 41, 51, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 121, 121, 122, 122, 122, 123,

In [0]:
a= [1, 2]

for i in a:
  print(i)

In [0]:
a=[1 ,2, 3, 4]
b=[2,3]
d=[2, 4, 5]
c = set(a) - set(d)

print(c)
e = set(c) - set(d)
print(c)
print(e)

f = set(e) - set(a)
print(type(f))
print(f)
if not f:
  print(not f)

{1, 3}
{1, 3}
{1, 3}
<class 'set'>
set()
True


In [0]:
t = [1, 1]

t.append(1,2)
print(t)

TypeError: ignored

In [0]:
t_id = [1, 1, 1, 1, 2, 2, 2, 2, 2]

d1 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d2 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d3 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d4 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d5 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d6 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d7 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d8 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d9 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")

test_data = [d1, d2, d3, d4, d5, d6, d7, d8, d9]

# Count odd numbers in the list
count = sum(map(lambda x : x == test_data[0], test_data))
 
a = sum(test_data == test_data for p in t_id)

print('Count of odd numbers in a list : ', count)

print(a)



Count of odd numbers in a list :  4
9


In [0]:
import datetime
t_space = [1, 1, 1, 3, 3, 3, 3, 3, 5]

d1 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d2 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d3 = datetime.datetime.strptime("01/09/2017", "%d/%m/%Y")
d4 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d5 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d6 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d7 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d8 = datetime.datetime.strptime("03/09/2017", "%d/%m/%Y")
d9 = datetime.datetime.strptime("05/09/2017", "%d/%m/%Y")

print(t_space.count(t_space[0]))
test_date = [d1, d2, d3, d4, d5, d6, d7, d8, d9]

# Count odd numbers in the list
count = sum(map(lambda x : x == r_space_id[i], t_space))
 
print('Count of odd numbers in a list : ', count)


In [0]:
fake_start_date = "01/09/2017" #str
real = "01/09/2017" #str

date = datetime.datetime.strptime(fake_start_date, "%d/%m/%Y")
next_date = date + datetime.timedelta(days=1)

real_date = datetime.datetime.strptime(real, "%d/%m/%Y")

print(date == real_date)
print (next_date)

True
2017-09-02 00:00:00


In [0]:
import csv
import datetime
with open('20191203_spaces_fix_locate.csv', newline='') as space_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  spaces_info = csv.DictReader(space_file)
  
  # 以迴圈讀取指定欄位
  locate_id = []; space_id = []; price = []; size = []; zipcode = [];
  open_day = []; close_day = []; open_time = []; close_time = [];
  for i in spaces_info:
    locate_id.append(i['locate_id'])
    space_id.append(i['space_id'])
    price.append(i['price'])
    size.append(i['size'])
    zipcode.append(i['zipcode'])
    open_day.append(i['open_day'])
    close_day.append(i['close_day'])
    open_time.append(i['open_time'])
    close_time.append(i['close_time'])
  space_file.close()

#space_id = list(map(int, space_id))
price = list(map(int, price))
size = list(map(int, size))
zipcode = list(map(int, zipcode))
'''
print(open_time)
print(type(open_time))
print(open_time[1])
print(type(open_time[1]))
'''



with open('20191204_test_real_order.csv', newline='', encoding='big5') as test_file:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary
  test_info = csv.DictReader(test_file)
  
  # 以迴圈讀取指定欄位
  r_space_id = []; r_date = []; r_start = []; r_end = [];
  for i in test_info:
    r_space_id.append(i['space_id'])
    r_date.append(i['date'])
    r_start.append(i['start'])
    r_end.append(i['end'])
  test_file.close()

r_space_id = list(map(int, r_space_id))

#print(open_time) #list
print(open_time[1]) #str
print(close_time[1]) #str

#print(r_date) #list
print(r_date[1]) #str

test = datetime.datetime.strptime(r_date[1], "%d/%m/%y")
print(test)

'''

format = '%H:%M'
start_Time = datetime.datetime.strptime(open_time[1], format)
end_Time = datetime.datetime.strptime(close_time[1], format)

diff = end_Time - start_Time
#diff = datetime.datetime.strptime(str(diff), '%H:%M:%S')

print(diff)
diff = str(diff)
print(type(diff))

'''



In [0]:
results = ['1', '2', '3']
results = list(map(int, results))
print(results)

In [0]:
import csv
with open('test.csv', 'w', newline='') as csvfile:
    testwriter = csv.writer(csvfile)
    start = 9 
    #start = '09:00:00'

    print('%02d' % start + ':00:00')
    testwriter.writerow(['start', 'end'])
    for i in range(0, 4):
      testwriter.writerow(['%02d' % start + ':00:00', '%02d' % (start+3) + ':00:00'])
      start += 3

In [0]:
import time
#named_tuple = time.localtime() # get struct_time
time_string = "1 September, 2017"
result = time.strptime(time_string, "%d %B, %Y")

test = result
time_string = time.strftime("%d/%m/%Y", test)
print(time_string)
print(type(test))
print(test[2]+1)



In [0]:
start = '09:00:00'
aa = 9

print('%02d' % aa)

aa += 3

print('%02d' % aa + ':00:00')


In [0]:
space_id = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,31,41,51,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187]

p =[1, 4, 8, 2]
h = 3
o_p = h * p
print(type(o_p))


for i in space_id:
  print(i, type(i))

